In [1]:
pip install kornia


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinu

In [2]:
!git clone https://github.com/GUILEEEE/deep-image-prior
!mv deep-image-prior/* ./

Cloning into 'deep-image-prior'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 303 (delta 1), reused 5 (delta 0), pack-reused 289
Receiving objects: 100% (303/303), 25.11 MiB | 20.04 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [3]:
%%writefile ./denoising_dipvbtvrdorder.py

import os
import numpy as np
import sys
#sys.path.insert(1, '/kaggle/input/high-oder-denoising/DIP_HighOrderVBTVDenoising')
from models import *
from models.vbtv_1 import*
from utils.sr_utils import*

import torch
import torch.optim

import kornia

from utils.denoising_utils import *
#from utils.blur_utils import *
from skimage.metrics import peak_signal_noise_ratio as psnr

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

imsize = -1
enforse_div32 = 'CROP' # we usually need the dimensions to be divisible by a power of two (32 in this case)


# Load clean image
k=3

fname = f'/content/data/denoising/F16_GT.png'
img_pil = crop_image(get_image(fname,-1)[0], d=32)
img_np = pil_to_np(img_pil)

# Load noisy image
fname = f'/content/data/denoising/F16_GT.png'
img_noisy_pil = crop_image(get_image(fname,-1)[0], d=32)
img_noisy_np = pil_to_np(img_noisy_pil)
img_noisy_torch = np_to_torch(img_noisy_np).type(dtype)


red = torch.narrow(img_noisy_torch,1,0,1) #[1x1xHxW]
red = torch.squeeze(red) #[HxW]

green = torch.narrow(img_noisy_torch,1,1,1) #[1x1xHxW]
green = torch.squeeze(green) #[HxW]

blue = torch.narrow(img_noisy_torch,1,2,1) #[1x1xHxW]
blue = torch.squeeze(blue) #[HxW]

diff_red_x = derivative_central_x_greylevel(red,dtype)     #[HxW]
diff_green_x = derivative_central_x_greylevel(green,dtype) #[HxW]
diff_blue_x = derivative_central_x_greylevel(blue,dtype)   #[HxW]

diff_red_y = derivative_central_y_greylevel(red,dtype)     #[HxW]
diff_green_y = derivative_central_y_greylevel(green,dtype) #[HxW]
diff_blue_y = derivative_central_y_greylevel(blue,dtype)   #[HxW]

diff_red_xx = derivative_xx_greylevel(red,dtype) #[HxW]
diff_red_xy = derivative_xy_greylevel(red,dtype) #[HxW]
diff_red_yy = derivative_yy_greylevel(red,dtype) #[HxW]

diff_green_xx = derivative_xx_greylevel(green,dtype) #[HxW]
diff_green_xy = derivative_xy_greylevel(green,dtype) #[HxW]
diff_green_yy = derivative_yy_greylevel(green,dtype) #[HxW]

diff_blue_xx = derivative_xx_greylevel(blue,dtype) #[HxW]
diff_blue_xy = derivative_xy_greylevel(blue,dtype) #[HxW]
diff_blue_yy = derivative_yy_greylevel(blue,dtype) #[HxW]

diff_red_xxx = derivative_xxx_greylevel(red,dtype) #[HxW]
diff_red_xxy = derivative_xxy_greylevel(red,dtype) #[HxW]
diff_red_xyy = derivative_xyy_greylevel(red,dtype) #[HxW]
diff_red_yyy = derivative_yyy_greylevel(red,dtype) #[HxW]

diff_green_xxx = derivative_xxx_greylevel(green,dtype) #[HxW]
diff_green_xxy = derivative_xxy_greylevel(green,dtype) #[HxW]
diff_green_xyy = derivative_xyy_greylevel(green,dtype) #[HxW]
diff_green_yyy = derivative_yyy_greylevel(green,dtype) #[HxW]

diff_blue_xxx = derivative_xxx_greylevel(blue,dtype) #[HxW]
diff_blue_xxy = derivative_xxy_greylevel(blue,dtype) #[HxW]
diff_blue_xyy = derivative_xyy_greylevel(blue,dtype) #[HxW]
diff_blue_yyy = derivative_yyy_greylevel(blue,dtype) #[HxW]


# Setup
INPUT = 'noise'
pad = 'reflection'
OPT_OVER='net'

reg_noise_std = 1./30.
LR = 0.01

OPTIMIZER = 'adam'
exp_weight = 0.99

num_iter = 8000
input_depth = 32

Lambda1 = 0.05
Lambda2 = 0.0001
Lambda3 = 0.#000001

gamma = 0. #1. #gamma=0 for Riemannian and Euclidean quadruplets
beta = 3000. #beta=0 for Euclidean quadruplet
epsilon = 0.0001

maxpsnravg = 0.
iter_maxpsnravg=0.
psnr_avg_gt = 0.

A = np.zeros(num_iter)
B = np.zeros(num_iter)

c=img_pil.size[1]
d=img_pil.size[0]

full_net = VectorBundleTotalVariationRplusRplusRplus3rdorderDeblurring(input_depth, pad, red, green, blue, diff_red_x,diff_green_x,diff_blue_x, diff_red_y,diff_green_y,diff_blue_y,diff_red_xx,diff_red_xy,diff_red_yy,diff_green_xx,diff_green_xy,diff_green_yy,diff_blue_xx,diff_blue_xy,diff_blue_yy,diff_red_xxx, diff_red_xxy, diff_red_xyy, diff_red_yyy, diff_green_xxx, diff_green_xxy, diff_green_xyy, diff_green_yyy, diff_blue_xxx, diff_blue_xxy, diff_blue_xyy, diff_blue_yyy, gamma, beta, epsilon, height=c, width=d, upsample_mode='bilinear' ).type(dtype)

net_input = get_noise(input_depth, INPUT, (c, d)).type(dtype).detach()

# Compute number of parameters
s  = sum([np.prod(list(p.size())) for p in full_net.parameters()])

# Loss
mse = torch.nn.MSELoss().type(dtype)
mae = torch.nn.L1Loss().type(dtype)

net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()
out_img_avg = img_noisy_np

i = 0
def closure():

	global i, exp_weight, out_img_avg, net_input, psnr_avg_gt,maxpsnravg,iter_maxpsnravg, maxpsnravg_image

	net_input = net_input_saved + (noise.normal_() * reg_noise_std)

	net_output, regularizer1,regularizer2, regularizer3 = full_net(net_input)


	loss_dataterm = mse(net_output,img_noisy_torch)
	loss_regularizer1 = mae(regularizer1,torch.zeros(1,c,d).type(dtype))
	loss_regularizer2 = mae(regularizer2,torch.zeros(1,c,d).type(dtype))
	loss_regularizer3 = mae(regularizer3,torch.zeros(1,c,d).type(dtype))

	total_loss = loss_dataterm + Lambda1*loss_regularizer1+Lambda2*loss_regularizer2+Lambda3*loss_regularizer3
	total_loss.backward(retain_graph=True)

	out_img_avg = out_img_avg * exp_weight + net_output.detach().cpu().numpy()[0] * (1 - exp_weight)
	psnr_avg_gt = psnr(img_np, out_img_avg)
	B[i]=psnr_avg_gt

	if psnr_avg_gt > maxpsnravg:
		maxpsnravg = psnr_avg_gt
		iter_maxpsnravg = i
		maxpsnravg_image = out_img_avg

		print(iter_maxpsnravg)
		print(maxpsnravg)

	if i==num_iter-1:
		print("final PSNR:", psnr_avg_gt)

	i += 1

	return total_loss

p = get_params(OPT_OVER, full_net, net_input, input_depth)
optimize(OPTIMIZER, p, closure, LR, num_iter)

out_img_maxpsnr_pil = np_to_pil(maxpsnravg_image)
out_img_maxpsnr_pil.save(f'./denoised_kodim{k}_maxpsnravg.png')
out_img_avg_pil = np_to_pil(out_img_avg)
out_img_avg_pil.save(f'./denoised_kodim{k}.png')
np.save(f'./psnr_avg_kodim{k}.npy',B)

Writing ./denoising_dipvbtvrdorder.py


In [4]:
!python denoising_dipvbtvrdorder.py

Starting optimization with ADAM
0
51.69663986191965
final PSNR: 37.94272404483701
